In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import mne
import scipy as sp
from scipy.signal import butter, sosfiltfilt
from EMG_analysis_bachelor.functions_for_pipeline import plot_channel_overview, normalize_emg, create_normalized_df, filtered
from functions_for_pipeline import create_raw_df, plot_raw_signals, notch_filter, bandpass_filter, create_filtered_df, plot_filtered_signals
import matplotlib as plt

In [20]:
EMG_only_test = mne.io.read_raw_ant("C:/Users/User/Documents/bachelorarbeit/data/EMG/"
                                   "test_first_2025-04-07_14-10-04_forearm_upperSide.cnt", preload=True)
EMG_ACC_data = mne.io.read_raw_ant("C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/"
                                   "PTB_measurement_14.04/Bonato_Federico_2025-04-14_13-02-56.cnt", preload=True)
custom_order_names = ["BIP6", "BIP1", "BIP2", "BIP7", "BIP8"]
#custom_order_names = ["BIP7", "BIP8"]
location = {"BIP7":"left forearm",
            "BIP8": "left delt",
            "BIP1" : "Charité ACC : y",
            "BIP2" : "Charité ACC : z",
            "BIP6" : "Charité ACC : x"}
EMG = ["BIP7", "BIP8"]
ACC = ["BIP6", "BIP1", "BIP2"]

data, times = EMG_ACC_data[custom_order_names, : ]
emg, times = EMG_ACC_data[EMG, :]
acc, times = EMG_ACC_data[ACC, :]
acc[0] *= -1
data[0] *= -1

Reading ANT file C:/Users/User/Documents/bachelorarbeit/data/EMG/test_first_2025-04-07_14-10-04_forearm_upperSide.cnt
All 32 EEG channels are referenced to CPz.
Reading 0 ... 15959  =      0.000 ...    31.918 secs...
Reading ANT file C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/PTB_measurement_14.04/Bonato_Federico_2025-04-14_13-02-56.cnt
Reading 0 ... 583854  =      0.000 ...   583.854 secs...


C:\Users\User\AppData\Local\Temp\ipykernel_16492\327492924.py:3: RuntimeWarning: All EEG channels are not referenced to the same electrode.
  EMG_ACC_data = mne.io.read_raw_ant("C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/"
C:\Users\User\AppData\Local\Temp\ipykernel_16492\327492924.py:3: RuntimeWarning: Unit unit not recognized, not scaling. Please report the unit on a github issue on https://github.com/mne-tools/mne-python.
  EMG_ACC_data = mne.io.read_raw_ant("C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/"


In [17]:
data[0]

array([0.40625   , 0.40625   , 0.40625   , ..., 0.34765625, 0.34765625,
       0.34765625])

In [18]:
acc[0]

array([0.40625   , 0.40625   , 0.40625   , ..., 0.34765625, 0.34765625,
       0.34765625])

In [7]:
# creating raw df
raw_df = create_raw_df(data, custom_order_names, times)
raw_df.head(2)

,BIP7,BIP8,Time (s)
0,0.00756,-0.150938,0.000
1,0.00762,-0.147162,0.002


In [ ]:
# plotting the raw signals
plot_channel_overview(custom_order_names, raw_df,"raw_signals", location, EMG)

In [8]:
# apply notch filter
notched_data = notch_filter(data, [50, 100, 150], 1000)
notched_data[0]

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



array([0.00756035, 0.00760524, 0.00772463, ..., 0.00867938, 0.00874303,
       0.00874335])

In [ ]:
notched_and_filtered_data = bandpass_filter(notched_data, 1000, 20, 450)

In [12]:
import matplotlib.pyplot as plt

sfreq = 1000 
duration = 1.0  
n_samples = int(sfreq * duration)
times = np.arange(n_samples) / sfreq 

freq = 9  # Hz
signal = np.sin(2 * np.pi * freq * times)

custom_order_names = ["BIP6", "BIP1", "BIP2", "BIP7", "BIP8"]
EMG = ["BIP7"]
ACC = ["BIP1"]

df = pd.DataFrame({
    "BIP1": signal,
    "BIP7": signal
})

plt.figure(figsize=(10, 4))
plt.plot(times, df["BIP1"], label="BIP 1")
plt.plot(times, df["BIP7"], label="BIP 7", linestyle='--', alpha=0.6)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("9 Hz Sinus Signal (identisch in beiden Spalten)")
plt.legend()
plt.tight_layout()
plt.show()

filtered_df = df.copy()
for col in custom_order_names:
    if col in ACC:
        col_data = df[col].to_numpy()
        col_filtered = mne.filter.filter_data(col_data, sfreq=1000, l_freq=1, h_freq=10)
        filtered_df[col] = col_filtered
    if col in EMG:
        col_data = df[col].to_numpy()
        col_filtered = mne.filter.filter_data(col_data, sfreq=1000, l_freq=30, h_freq=480)
        filtered_df[col] = col_filtered
    else:
        continue

plt.figure(figsize=(10, 4))
plt.plot(times, filtered_df["BIP1"], label="BIP 1")
plt.plot(times, filtered_df["BIP7"], label="BIP 7", linestyle='--', alpha=0.6)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("filtered Signal")
plt.legend()
plt.tight_layout()
plt.show()

Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 2.50 Hz (-6 dB cutoff frequency: 11.25 Hz)
- Filter length: 3301 samples (3.301 s)

Setting up band-pass filter from 30 - 4.8e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 480.00 Hz
- Upper transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 490.0

C:\Users\User\AppData\Local\Temp\ipykernel_10476\2694037808.py:36: RuntimeWarning: filter_length (3301) is longer than the signal (1000), distortion is likely. Reduce filter length or filter a longer signal.
  col_filtered = mne.filter.filter_data(col_data, sfreq=1000, l_freq=1, h_freq=10)
